In [ ]:
import ast 
from tqdm.auto import tqdm
import pandas as pd
import csv


Открываем списки городов перемещений:

In [ ]:
df = pd.read_csv('people.csv', sep='\t')

cts = list(df['City'])
uns = list(df['Universities'])
schs = list(df['Schools'])


Делаем списки из _id_ городов каждого перемещения:

In [ ]:
def remake(l):
    new = []
    for each in l:
        if  type(each) == str:
            a = each.split(', ')
            new.append(a)
        else:
            try:

                new.append(int(each))
            except:
                ValueError
                new.append('')
    return new


In [4]:
uns = remake(uns)
schs = remake(schs)
cts = remake(cts)


Находим следующие перемещения:
1. Из одной школы в другую: если в списке городов школ есть различные города;
2. Из школы в университет: если последний город школы не совпадает с первым городом университета;
3. Из одного университета в другой: если в списке городов университетов есть различные города;
4. Перемещение из университета в настоящий город проживания (по работе или иным причинам): если последний город университета/школы не совпадает с настоящим городом проживания;
5. Полный маршрут: не повторяющиеся подряд города из всех списков;
6. Число людей, не совершавших никаких перемещений.

In [ ]:
def search_transfers(school_cities, university_cities, city_ids):
    
    routes_work = []
    routes_school = []
    routes_schtouni = []
    routes_university = []
    full_route = []
    no_transfers = 0
    
    for i in range(len(city_ids)):
        b = []
        if school_cities[i] and school_cities[i] != '-':
            b.extend(school_cities[i])
            if len(school_cities[i]) > 1:
                routes_school.append(', '.join(school_cities[i]))
            
        if university_cities[i] and university_cities[i] != '-':
            if len(university_cities[i]) > 1:
                routes_university.append(', '.join(university_cities[i]))
            if b:
                if university_cities[i][0] != b[-1]:
                    routes_schtouni.append(', '.join([b[-1], university_cities[i][0]]))
                    b.extend(university_cities[i])
            else:
                b.extend(university_cities[i])
            
        if city_ids[i] and city_ids[i] != '-':
            if b:
                if str(city_ids[i]) != b[-1]:
                    routes_work.append(', '.join([b[-1], str(city_ids[i])]))
                    b.append(str(city_ids[i]))
            else:
                b.append(str(city_ids[i]))
        if len(b) > 1:
            full_route.append(', '.join(b))
        elif len(b) == 1:
            no_transfers += 1

    return routes_work, routes_school, routes_schtouni, routes_university, full_route, no_transfers


In [ ]:
routes_work, routes_school, routes_schtouni, routes_university, full_route, no_transfers = search_transfers(schs, uns, cts)
print(len(routes_work), len(routes_school), len(routes_schtouni), len(routes_university), len(full_route), no_transfers)

# no sex

Считаем повторяющиеся маршруты и создаем словарь вида _'маршрут: число таких же'_:

In [ ]:
def get_number_of_similar_routes(list_of_routes):
    routes_nums = {}
    for route in list_of_routes:
        try:
            routes_nums[route] += 1
        except:
            routes_nums[route] = 1
    return routes_nums


In [ ]:
def get_unique_routes(route_list):
    unique = list(set(route_list))
    return unique


In [ ]:
unique_routes_work, unique_routes_school, unique_routes_schtouni, unique_routes_university, unique_full_route = get_unique_routes(routes_work), get_unique_routes(routes_school), get_unique_routes(routes_schtouni), get_unique_routes(routes_university), get_unique_routes(full_route)
num_routes_work, num_routes_school, num_routes_schtouni, num_routes_university, num_full_route = get_number_of_similar_routes(routes_work), get_number_of_similar_routes(routes_school), get_number_of_similar_routes(routes_schtouni), get_number_of_similar_routes(routes_university), get_number_of_similar_routes(full_route)


Записываем получившуюся информацию в csv-файл для дальнейшей обработки (файлов получится несколько, содеиним их в join_routes.pynb):

In [10]:
with open('all_routes.csv', 'a', encoding='utf-8') as csv_file:
    file_writer = csv.writer(csv_file, delimiter = "\t")
    file_writer.writerow(['type', 'route', 'num'])
    for i in range(len(unique_routes_work)):
        file_writer.writerow(['work', unique_routes_work[i], num_routes_work[unique_routes_work[i]]])
    for i in range(len(unique_routes_school)):
        file_writer.writerow(['school', unique_routes_school[i], num_routes_school[unique_routes_school[i]]])
    for i in range(len(unique_routes_schtouni)):
        file_writer.writerow(['school_to_university', unique_routes_schtouni[i], num_routes_schtouni[unique_routes_schtouni[i]]])
    for i in range(len(unique_routes_university)):
        file_writer.writerow(['university', unique_routes_university[i], num_routes_university[unique_routes_university[i]]])
    for i in range(len(unique_full_route)):
        file_writer.writerow(['full', unique_full_route[i], num_full_route[unique_full_route[i]]])
    file_writer.writerow(['no_transfers', '', no_transfers])
    